In [5]:
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
from bs4 import BeautifulSoup
import lxml
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from time import sleep
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from tqdm import tqdm
import pandas_profiling
import re

%matplotlib inline

## Predicting Data Roles
1. Scrape data with regards to:
   - salary
   - job title
   - required skills
   - location (city & area)
   - company
   


In [460]:
homepage = 'https://www.seek.com.au/data-jobs/in-All-Australia?salaryrange=60000-999999&salarytype=annual'
response = requests.get(homepage)
print(response)

<Response [200]>


In [481]:
driver = webdriver.Chrome(executable_path="/Users/elkehansen/Documents/general_assembly/Unit-3-Restructured/04_WebScraping/dsi-unit-3.33-python-webscraping_opentable-lab/chromedriver/chromedriver")
driver.get('https://www.seek.com.au/data-jobs/in-All-Australia?salaryrange=60000-999999&salarytype=annual')
sleep(1)
html = driver.page_source

In [411]:
len(html)

332531

In [412]:
soup = BeautifulSoup(html, 'lxml')

### Scraping within Links

In [476]:
### iterating through the first 749 pages of job ads to pull out the url of each job
links = []
for i in tqdm(range(1, 750)):
    url = 'https://www.seek.com.au/data-jobs/in-All-Australia?page=' + str(i) + '&salaryrange=60000-999999&salarytype=annual'
    res = requests.get(url)
    soup = BeautifulSoup(markup = res.text, features = 'lxml')
    jobs = soup.find_all('a', attrs={'data-automation': 'jobTitle'})
    for job in jobs:
        job_link = job.get('href')
        links.append(job_link)

100%|██████████| 749/749 [04:29<00:00,  3.91it/s]


In [482]:
### Using Selenium to cycle through the links to pull out the salary estimate information.
### This takes quite a bit longer for processing to occur so is not done with the other block of extractions
job_salary_estimates = []
for link in tqdm(links):
    url = 'https://www.seek.com.au' + str(link) + '&type=promoted'
    driver.get(url)
    sleep(1)
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'lxml')
    try:
        salary_est = soup2.find('div', attrs={'class':'lmis-cg-salary-bottom'})
        job_salary_estimates.append(salary_est.text)
    except:
        job_salary_estimates.append(None)

100%|██████████| 4370/4370 [2:40:50<00:00,  2.21s/it]  


##### removal of index 637 due to causing errors in main scraping code BEFORE putting in a try/except method to skip theses

In [ ]:
links.pop(637)

In [497]:
job_salary_estimates.pop(637)

'\n$95K\n$160K\n$200K\n'

###### Code snippets used for the trial and error of sections of the main scraping code before running on all pages

In [425]:

url = 'https://www.seek.com.au/job/39544932?type=standard&searchrequesttoken=5840dcba-b9cc-49d2-83ee-790f26ce4858'
res = requests.get(url)
soup = BeautifulSoup(html2, 'lxml')
# try:
main_info = soup.find('div', attrs={'class':'_2njvnpA Pdwn1mb'})
print(main_info)
# except:
#     print(None)

<div class="_2njvnpA Pdwn1mb"><section aria-labelledby="jobInfoHeader" role="region"><span class="Eadjc1o"><h1 id="jobInfoHeader">Job Information</h1></span><dl><dt><span class="Eadjc1o">Job Listing Date</span></dt><dd data-automation="job-detail-date"><span class="_3FrNV7v _3PZrylH E6m4BZb"><span class="lwHBT6d">1 Aug 2019</span></span></dd><dt><span class="Eadjc1o">Location</span></dt><dd><span class="_3FrNV7v _3PZrylH E6m4BZb"><span class=""><strong class="lwHBT6d">Melbourne</strong><span class="eBOHjGN"><span class="_2njvnpA">, </span>CBD &amp; Inner Suburbs</span></span></span></dd><dt><span class="Eadjc1o">Work Type</span></dt><dd data-automation="job-detail-work-type"><span class="_3FrNV7v _3PZrylH E6m4BZb"><span class="lwHBT6d">Full Time</span></span></dd><div><dt><span class="Eadjc1o">Classification</span></dt><dd><span class="_3FrNV7v _3PZrylH E6m4BZb"><span class=""><strong class="lwHBT6d">Accounting</strong><span class="eBOHjGN"><span class="_2njvnpA">, </span>Analysis &amp

In [427]:
## need to be pulled then split by indexing
main_info.find_all('strong', attrs={'class':'lwHBT6d'})

[<strong class="lwHBT6d">Melbourne</strong>,
 <strong class="lwHBT6d">Accounting</strong>]

In [428]:
## need to be pulled then split by indexing
main_info.find_all('span', attrs={'class':'eBOHjGN'})

[<span class="eBOHjGN"><span class="_2njvnpA">, </span>CBD &amp; Inner Suburbs</span>,
 <span class="eBOHjGN"><span class="_2njvnpA">, </span>Analysis &amp; Reporting</span>]

In [429]:
## need to be pulled then split by indexing

main_info.find_all('span', attrs={'class':'lwHBT6d'})


[<span class="lwHBT6d">1 Aug 2019</span>,
 <span class="lwHBT6d">Full Time</span>]

### Main scraping Code
1. Create an empty list for each attribute
2. iterate through each link and setup the soup 
3. Outer try/except code to skip over pages which have been removed since the selenium scrape and throwing errors
4. Individualised pieces of code for each attribute. I have tried to keep this as clean as possible but there are a lot of different methods needed for each list.

In [505]:
salaries = []
industries = []
sub_industries = []
contracts = []
locations = []
proximity_to_locations = []
role_text = []
company_ratings = []
companies = []
titles = []

for link in tqdm(links):
    url = 'https://www.seek.com.au' + str(link) 
    res = requests.get(url)
    soup = BeautifulSoup(markup = res.text, features = 'lxml')
    
    try:
        title = soup.find('span', attrs={'data-automation': 'job-detail-title'})
        titles.append(title.text)
    
        try:
            company = soup.find('span', attrs={'data-automation': ['job-header-company-review-title','advertiser-name']})
            companies.append(company.text)
        except:
            companies.append(None)
    
        text_block = soup.find('div', attrs={'data-automation': 'mobileTemplate'})
        role_text.append(text_block.text)
    
        contract = soup.find('dd', attrs={'data-automation': 'job-detail-work-type'})
        contracts.append(contract.text)
    
        try:
            company_rating = soup.find('span', attrs={'class': '_1erK2ob'})
            company_ratings.append(company_rating.text)      
        except:
            company_ratings.append(None)
    
    
        # section of page containing several different variables
        main_info = soup.find('div', attrs={'class':'_2njvnpA Pdwn1mb'})
    
        location = main_info.find_all('strong', attrs={'class':'lwHBT6d'})[0]
        locations.append(location.text)
    
        industry = main_info.find_all('strong', attrs={'class':'lwHBT6d'})[1]
        industries.append(industry.text)
    
        salary = main_info.find_all('span', attrs={'class':'lwHBT6d'})[1]
        salaries.append(salary.text)
    
        sub_industry = main_info.find_all('span', attrs={'class':'eBOHjGN'})[-1]
        sub_industries.append(sub_industry.text)
   
        try: 
            proximity_to_location = main_info.find_all('span', attrs={'class':'eBOHjGN'})[-2]
            proximity_to_locations.append(proximity_to_location.text)
        except:
            proximity_to_locations.append(None)
    except:
        print(link)
        print(len(titles))
        pass
    
    





  0%|          | 0/4369 [00:00<?, ?it/s]



  0%|          | 1/4369 [00:00<23:35,  3.09it/s]



  0%|          | 2/4369 [00:00<22:03,  3.30it/s]



  0%|          | 3/4369 [00:00<20:53,  3.48it/s]



  0%|          | 4/4369 [00:01<19:37,  3.71it/s]



  0%|          | 5/4369 [00:01<19:13,  3.78it/s]



  0%|          | 6/4369 [00:01<18:58,  3.83it/s]



  0%|          | 7/4369 [00:01<18:25,  3.95it/s]



  0%|          | 8/4369 [00:02<17:56,  4.05it/s]



  0%|          | 9/4369 [00:02<19:00,  3.82it/s]



  0%|          | 10/4369 [00:02<18:33,  3.92it/s]



  0%|          | 11/4369 [00:02<18:42,  3.88it/s]



  0%|          | 12/4369 [00:03<18:38,  3.90it/s]



  0%|          | 13/4369 [00:03<17:54,  4.06it/s]



  0%|          | 14/4369 [00:03<17:31,  4.14it/s]



  0%|          | 15/4369 [00:03<17:46,  4.08it/s]



  0%|          | 16/4369 [00:04<19:21,  3.75it/s]



  0%|          | 17/4369 [00:04<19:11,  3.78it/s]



  0%|          | 18/4369 [00:04<18:14,  3.98it/s]



  0%|  

/job/39544932?type=standard&searchrequesttoken=5840dcba-b9cc-49d2-83ee-790f26ce4858
989






 23%|██▎       | 991/4369 [04:48<16:43,  3.37it/s]



 23%|██▎       | 992/4369 [04:49<17:29,  3.22it/s]



 23%|██▎       | 993/4369 [04:49<16:08,  3.49it/s]



 23%|██▎       | 994/4369 [04:49<14:57,  3.76it/s]



 23%|██▎       | 995/4369 [04:49<15:47,  3.56it/s]



 23%|██▎       | 996/4369 [04:50<20:27,  2.75it/s]



 23%|██▎       | 997/4369 [04:50<18:48,  2.99it/s]



 23%|██▎       | 998/4369 [04:50<17:40,  3.18it/s]



 23%|██▎       | 999/4369 [04:51<17:33,  3.20it/s]



 23%|██▎       | 1000/4369 [04:51<16:23,  3.43it/s]



 23%|██▎       | 1001/4369 [04:51<19:04,  2.94it/s]



 23%|██▎       | 1002/4369 [04:52<17:17,  3.25it/s]



 23%|██▎       | 1003/4369 [04:52<15:49,  3.55it/s]



 23%|██▎       | 1004/4369 [04:52<17:53,  3.14it/s]



 23%|██▎       | 1005/4369 [04:53<16:25,  3.41it/s]



 23%|██▎       | 1006/4369 [04:53<15:30,  3.61it/s]



 23%|██▎       | 1007/4369 [04:53<14:21,  3.90it/s]



 23%|██▎       | 1008/4369 [04:53<14:22,  3.90it/s]



 23%|██▎       

/job/39544438?type=standard&searchrequesttoken=4a0a6926-dd00-452d-a605-fa08f2b26974
1302






 30%|██▉       | 1305/4369 [06:22<13:41,  3.73it/s]



 30%|██▉       | 1306/4369 [06:22<13:23,  3.81it/s]



 30%|██▉       | 1307/4369 [06:22<13:18,  3.83it/s]



 30%|██▉       | 1308/4369 [06:23<16:10,  3.15it/s]



 30%|██▉       | 1309/4369 [06:23<14:29,  3.52it/s]



 30%|██▉       | 1310/4369 [06:23<14:27,  3.53it/s]



 30%|███       | 1311/4369 [06:23<13:55,  3.66it/s]



 30%|███       | 1312/4369 [06:24<13:18,  3.83it/s]



 30%|███       | 1313/4369 [06:24<14:00,  3.64it/s]



 30%|███       | 1314/4369 [06:24<13:25,  3.79it/s]



 30%|███       | 1315/4369 [06:24<13:07,  3.88it/s]



 30%|███       | 1316/4369 [06:25<12:58,  3.92it/s]



 30%|███       | 1317/4369 [06:25<12:33,  4.05it/s]



 30%|███       | 1318/4369 [06:25<12:31,  4.06it/s]



 30%|███       | 1319/4369 [06:25<12:20,  4.12it/s]



 30%|███       | 1320/4369 [06:26<12:44,  3.99it/s]



 30%|███       | 1321/4369 [06:26<13:03,  3.89it/s]



 30%|███       | 1322/4369 [06:26<13:13,  3.84it/s]



 30%|█

...





 65%|██████▍   | 2819/4369 [13:53<07:11,  3.59it/s]



 65%|██████▍   | 2820/4369 [13:53<06:59,  3.69it/s]



 65%|██████▍   | 2821/4369 [13:53<07:16,  3.55it/s]



 65%|██████▍   | 2822/4369 [13:54<07:02,  3.67it/s]



 65%|██████▍   | 2823/4369 [13:54<06:48,  3.79it/s]



 65%|██████▍   | 2824/4369 [13:54<06:38,  3.87it/s]



 65%|██████▍   | 2825/4369 [13:54<06:30,  3.95it/s]



 65%|██████▍   | 2826/4369 [13:55<06:26,  3.99it/s]



 65%|██████▍   | 2827/4369 [13:55<06:40,  3.85it/s]



 65%|██████▍   | 2828/4369 [13:55<06:24,  4.01it/s]



 65%|██████▍   | 2829/4369 [13:55<06:32,  3.92it/s]



 65%|██████▍   | 2830/4369 [13:56<06:19,  4.06it/s]



 65%|██████▍   | 2831/4369 [13:56<06:09,  4.16it/s]



 65%|██████▍   | 2832/4369 [13:56<06:22,  4.02it/s]



 65%|██████▍   | 2833/4369 [13:56<06:22,  4.01it/s]



 65%|██████▍   | 2834/4369 [13:57<06:13,  4.11it/s]



 65%|██████▍   | 2835/4369 [13:57<06:12,  4.12it/s]



 65%|██████▍   | 2836/4369 [13:57<08:13,  3.11it/s]



 65%|█

/job/39470863?type=standout&searchrequesttoken=f371f416-3b6c-4190-ae6d-939166ccd6b0
3665






 84%|████████▍ | 3671/4369 [18:00<02:57,  3.92it/s]



 84%|████████▍ | 3672/4369 [18:01<03:31,  3.30it/s]



 84%|████████▍ | 3673/4369 [18:01<03:30,  3.30it/s]



 84%|████████▍ | 3674/4369 [18:01<03:32,  3.27it/s]



 84%|████████▍ | 3675/4369 [18:02<03:25,  3.38it/s]



 84%|████████▍ | 3676/4369 [18:02<03:17,  3.51it/s]



 84%|████████▍ | 3677/4369 [18:02<03:09,  3.65it/s]



 84%|████████▍ | 3678/4369 [18:02<03:00,  3.83it/s]



 84%|████████▍ | 3679/4369 [18:03<02:55,  3.92it/s]



 84%|████████▍ | 3680/4369 [18:03<02:53,  3.98it/s]



 84%|████████▍ | 3681/4369 [18:03<02:58,  3.86it/s]



 84%|████████▍ | 3682/4369 [18:03<02:56,  3.90it/s]



 84%|████████▍ | 3683/4369 [18:04<02:49,  4.04it/s]



 84%|████████▍ | 3684/4369 [18:04<02:42,  4.20it/s]



 84%|████████▍ | 3685/4369 [18:04<02:57,  3.84it/s]



 84%|████████▍ | 3686/4369 [18:04<02:52,  3.97it/s]



 84%|████████▍ | 3687/4369 [18:05<03:49,  2.97it/s]



 84%|████████▍ | 3688/4369 [18:05<03:30,  3.24it/s]



 84%|█

/job/39514769?type=promoted&searchrequesttoken=3c3e11b6-4b5b-4d85-abc1-bed46b375e20
3734






 86%|████████▌ | 3741/4369 [18:20<03:01,  3.46it/s]



 86%|████████▌ | 3742/4369 [18:20<02:51,  3.65it/s]



 86%|████████▌ | 3743/4369 [18:21<02:41,  3.87it/s]



 86%|████████▌ | 3744/4369 [18:21<02:41,  3.87it/s]



 86%|████████▌ | 3745/4369 [18:21<02:38,  3.94it/s]



 86%|████████▌ | 3746/4369 [18:21<02:46,  3.75it/s]



 86%|████████▌ | 3747/4369 [18:22<02:41,  3.86it/s]



 86%|████████▌ | 3748/4369 [18:22<02:37,  3.95it/s]



 86%|████████▌ | 3749/4369 [18:22<02:36,  3.97it/s]



 86%|████████▌ | 3750/4369 [18:22<02:31,  4.08it/s]



 86%|████████▌ | 3751/4369 [18:23<02:29,  4.15it/s]



 86%|████████▌ | 3752/4369 [18:23<02:26,  4.22it/s]



 86%|████████▌ | 3753/4369 [18:23<02:24,  4.26it/s]



 86%|████████▌ | 3754/4369 [18:23<02:31,  4.06it/s]



 86%|████████▌ | 3755/4369 [18:24<02:30,  4.07it/s]



 86%|████████▌ | 3756/4369 [18:24<02:28,  4.12it/s]



 86%|████████▌ | 3757/4369 [18:24<02:24,  4.22it/s]



 86%|████████▌ | 3758/4369 [18:24<02:35,  3.93it/s]



 86%|█

#### Remove from links and job_salary_estimates:
* /job/39544932?type=standard&searchrequesttoken=5840dcba-b9cc-49d2-83ee-790f26ce4858
* 989
* /job/39544438?type=standard&searchrequesttoken=4a0a6926-dd00-452d-a605-fa08f2b26974
* 1302
* /job/39470863?type=promoted&searchrequesttoken=d56d1a2d-da22-43ba-b3ab-aaf0d143d8ac
* 2376
* /job/39470863?type=promoted&searchrequesttoken=15dea744-1ff2-42b5-b74c-efdcc4add1eb
* 2814
* /job/39470863?type=standout&searchrequesttoken=f371f416-3b6c-4190-ae6d-939166ccd6b0
* 3665
* /job/39514769?type=promoted&searchrequesttoken=3c3e11b6-4b5b-4d85-abc1-bed46b375e20
* 3734

1. Check that the index matches the url
2. pop from both lists

##### This could have been turned into a simple function, however, there were only 6 links and because the scraping took so long, I wanted to do it manually to make certain it was done correctly

In [509]:
links[989]

'/job/39544932?type=standard&searchrequesttoken=5840dcba-b9cc-49d2-83ee-790f26ce4858'

In [510]:
links.pop(989)
job_salary_estimates.pop(989)

'\n$90K\n$130K\n$170K\n'

In [512]:
links[1302]

'/job/39544438?type=standard&searchrequesttoken=4a0a6926-dd00-452d-a605-fa08f2b26974'

In [513]:
links.pop(1302)
job_salary_estimates.pop(1302)

'\n$35K\n$90K\n$140K\n'

In [514]:
links[2376]

'/job/39470863?type=promoted&searchrequesttoken=d56d1a2d-da22-43ba-b3ab-aaf0d143d8ac'

In [516]:
links.pop(2376)
job_salary_estimates.pop(2376)

In [517]:
links[2814]

'/job/39470863?type=promoted&searchrequesttoken=15dea744-1ff2-42b5-b74c-efdcc4add1eb'

In [518]:
links.pop(2814)
job_salary_estimates.pop(2814)

In [519]:
links[3665]

'/job/39470863?type=standout&searchrequesttoken=f371f416-3b6c-4190-ae6d-939166ccd6b0'

In [520]:
links.pop(3665)
job_salary_estimates.pop(3665)

In [521]:
links[3734]

'/job/39514769?type=promoted&searchrequesttoken=3c3e11b6-4b5b-4d85-abc1-bed46b375e20'

In [522]:
links.pop(3734)
job_salary_estimates.pop(3734)

##### Check all the columns are the same length

In [523]:
print(len(links))
print(len(job_salary_estimates))

4363
4363


In [524]:
len(titles)

4363

In [525]:
len(proximity_to_locations)

4363

In [529]:
urls = []
for link in links:
    url = 'https://www.seek.com.au' + str(link)
    urls.append(url)

##### Create a dataframe containing all the scraped information

In [530]:
data = {"Job Title" : titles, 
        "Company" : companies,
        "Company Rating" : company_ratings,
        "Salary" : salaries,
        "Industry" : industries,
        "Sub Industry" : sub_industries,
        "City" : locations,
        "Proximity to city" : proximity_to_locations,
        "Contract Type" : contracts,
        "Role Description" : role_text,
        "Salary Estimators" : job_salary_estimates,
        "Url" : urls}

In [35]:
df = pd.DataFrame(data=data)

In [532]:
df.head()

,Job Title,Company,Company Rating,Salary,Industry,Sub Industry,City,Proximity to city,Contract Type,Role Description,Salary Estimators,Url
0,Digital Analyst,JORA,5.0 overall rating (16 employee reviews),Base + Super + Profit Share,Marketing & Communications,", Digital & Search Marketing",Melbourne,", CBD & Inner Suburbs",Full Time,About Jora Jora is an Australian based job sea...,\n$55K\n$90K\n$140K\n,https://www.seek.com.au/job/39576804?type=prom...
1,Data Analyst - Data Quality,RACWA,3.5 overall rating (109 employee reviews),Contract/Temp,Information & Communication Technology,", Business/Systems Analysts",Perth,", CBD, Inner & Western Suburbs",Contract/Temp,In light of our Member Centred Program and sh...,\n$35K\n$70K\n$140K\n,https://www.seek.com.au/job/39487729?type=prom...
2,Data & Reporting Analyst,Telecommunications Industry Ombudsman,2.9 overall rating (18 employee reviews),"$70,000 - $84,999",Information & Communication Technology,", Business/Systems Analysts",Melbourne,", CBD & Inner Suburbs",Full Time,24 Month Maximum Term role Competitive sala...,None,https://www.seek.com.au/job/39594776?type=stan...
3,Junior Data Analyst,RXP Group,None,Full Time,Information & Communication Technology,", Database Development & Administration",Sydney,", CBD, Inner West & Eastern Suburbs",Full Time,RXP Group (ASX: RXP) A Digital Services Partne...,\n$30K\n$110K\n$160K\n,https://www.seek.com.au/job/39597326?type=stan...
4,Data Analyst,Chandler Macleod Group,None,super,Information & Communication Technology,", Other",Sydney,", CBD, Inner West & Eastern Suburbs",Contract/Temp,One of our clients is looking for a Data Speci...,\n$30K\n$110K\n$160K\n,https://www.seek.com.au/job/39610701?type=stan...


In [26]:
df.shape

(4363, 13)

##### Save to csv for use in another notebook

In [27]:
df.to_csv(r'/Users/elkehansen/Documents/general_assembly/Project-4-Submission/Elke_Project_4/seek-data.csv')